In [ ]:
! git clone https://www.github.com/pafitis/thesis

Cloning into 'thesis'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 294 (delta 166), reused 202 (delta 75), pack-reused 0
Receiving objects: 100% (294/294), 54.41 MiB | 25.56 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [ ]:
! pip install spacy-nightly
! python -m spacy download en_core_web_trf
! pip install pickle5
! pip install SentencePiece

     |████████████████████████████████| 12.7 MB 189 kB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 9.1 MB 40.2 MB/s 
     |████████████████████████████████| 621 kB 47.8 MB/s 
     |████████████████████████████████| 456 kB 48.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.6 wh

# YOU NEED TO RESTART THE RUNTIME!

In [ ]:
% cd thesis

/content/thesis


In [ ]:
import sys
sys.path.append('thesis')

In [ ]:
import numpy as np
import pandas as pd
import pickle5 as pickle
import torch
import json

from helpers.classes import Collection
from tqdm import tqdm
from datetime import datetime
from time import time
from google.colab import files
from transformers import pipeline

from helpers.cloze_generation import \
    generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.language_modelling import \
    find_nth_substring, multitoken_prediction, check_model, run_language_model

from helpers.t5_language_model import \
    run_t5, check_t5

In [ ]:
clozes_df = pd.read_json('pickles/clozes_with_false_20210825.json')

In [ ]:
from torch.utils.data import DataLoader, Dataset

class ClozeDataset(Dataset):
    def __init__(self, clozes_df, mask_token = '[MASK]'):

        self.clozes_df = clozes_df
        self.cloze_text = clozes_df.cloze_text
        self.answer_text = clozes_df.answer_text
        self.ids = clozes_df.ids
        self.mask_token = mask_token
        self.masks = ["IDENTITYMASK", "NOUNPHRASEMASK", "NUMERICMASK", 
                  "PLACEMASK", "TEMPORALMASK", "THINGMASK"]

    def __len__(self):
        return len(self.cloze_text)

    def __getitem__(self, idx):
        cloze = self.cloze_text.iloc[idx]
        answer = self.answer_text.iloc[idx]
        cloze_id = self.ids.iloc[idx]

        cloze = [cloze.replace(x, self.mask_token) \
                 for x in self.masks if x in cloze][0]

        return cloze, answer, cloze_id

# BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
print('Initialising Bert...')

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
bert_model_name = 'bert-base-uncased'

unmasker = pipeline('fill-mask', model=bert_model_name)
# unmasker = pipeline('fill-mask', model=bert_model_name, device = 0)

Initialising Bert...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
dataset = ClozeDataset(clozes_df, mask_token = '[MASK]')
dataloader = DataLoader(dataset, batch_size=10, shuffle = False, pin_memory=True)

In [ ]:
list(clozes)

['In [MASK], approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be £1,313.9 billion; an increase of £42.8 billion (3.4%) compared with 2018.',
 'In 2019, approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be [MASK]; an increase of £42.8 billion (3.4%) compared with 2018.',
 'In 2019, approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be £1,313.9 billion; an increase of [MASK] (3.4%) compared with 2018.',
 'In 2019, approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be £1,313.9 billion; an increase of £42.8 billion ([MASK]) compared with 2018.',
 'In 2019, approximate gross value added at basic prices (aGVA) of the UK non-financial business economy was estimated to be £1,313.9 billion; an increase of £42.8 billion (3.4%) compared with [MASK].',
 'Tota

In [ ]:
# with open('results/bert_results_25august.json', 'w') as f:

#     for idx, batch in tqdm(enumerate(dataloader)):
#         clozes, answers, cloze_ids = batch[0], batch[1], batch[2]
#         # predictions = unmasker(clozes)
#         predictions = unmasker(list(clozes))
#         for item, answer, cloze_id in zip(predictions, answers, cloze_ids):
#             for pred in item:
#                 pred['answer'] = answer
#                 pred['ids'] = cloze_id
        

#         json.dump(predictions , f, ensure_ascii = False)
# files.download('results/bert_results_25august.json')

619it [17:31,  1.70s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
all_predictions = []
for idx, batch in tqdm(enumerate(dataloader)):
    clozes, answers, cloze_ids = batch[0], batch[1], batch[2]
    # predictions = unmasker(clozes)
    predictions = unmasker(list(clozes))
    for item, answer, cloze_id in zip(predictions, answers, cloze_ids):
        for pred in item:
            pred['answer'] = answer
            pred['ids'] = cloze_id
    
    all_predictions.append(predictions)

with open('results/bert_results_25august.json', 'w') as f:
    json.dump(all_predictions, f, ensure_ascii = False)

files.download('results/bert_results_25august.json')

619it [17:17,  1.68s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ROBERTA

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
print('Initialising Roberta...')
device = "cuda:0" if torch.cuda.is_available() else "cpu"

roberta_model_name = 'roberta-base'

# model = RobertaForMaskedLM.from_pretrained('roberta-base').to(device)
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

unmasker = pipeline('fill-mask', model=roberta_model_name, device = 0)

Initialising Roberta...


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
dataset = ClozeDataset(clozes_df, mask_token = '<mask>')
dataloader = DataLoader(dataset, batch_size=10, shuffle = False, pin_memory=True)

In [ ]:
with open('results/roberta_results_25august.json', 'w') as f:

    for idx, batch in tqdm(enumerate(dataloader)):
        clozes, answers, cloze_ids = batch[0], batch[1], batch[2]
        predictions = unmasker(clozes)
        for item, answer, cloze_id in zip(predictions, answers, cloze_ids):
            for pred in item:
                pred['answer'] = answer
                pred['ids'] = cloze_id
        

        json.dump(predictions , f, ensure_ascii = False)
files.download('results/roberta_results_25august.json')

619it [00:47, 13.01it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# T5

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer

print('Initialising T5...')
device = 'cuda:0' if torch.cuda.is_available else 'cpu'

t5_model_name = 't5-base'
num_beams, num_return_sequences, max_length = 200, 5, 10

config = T5Config.from_pretrained(
    t5_model_name)
model = T5ForConditionalGeneration.from_pretrained(
    t5_model_name, config=config).to(device)
tokenizer = T5Tokenizer.from_pretrained(
    t5_model_name, config=config)

Initialising T5...


In [ ]:
with open('results/t5_results_25august.json', 'w') as f:

    for idx, row in tqdm(clozes_df.iterrows()):
        cloze = row.cloze_text
        answer = row.answer_text
        id = row.ids

        predictions = check_t5(model, tokenizer, cloze,
                              num_beams, num_return_sequences,
                              max_length)
        row_result = []
        for sequence, token_str in predictions:
            entry = {'sequence':sequence,
                        'token_str': token_str, 
                        'answer': answer, 'ids': id}
            row_result.append(entry)

        json.dump(row_result , f, ensure_ascii = False)
files.download('results/t5_results_25august.json')

6184it [3:12:53,  1.87s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>